In [1]:
Dir.pwd

"/examples/notebooks"

In [2]:
command = "uo create --project-folder #{Dir.pwd}/URBANoptOSA"

"uo create --project-folder /examples/notebooks/URBANoptOSA"

In [3]:
output = `#{command}`

"\nCreating a new project folder...\nWeather data is provided for the example FeatureFile. Additional weather data files may be downloaded from energyplus.net/weather for free\nIf you use additional weather files, ensure they are added to the 'weather' directory. You will need to configure your mapper file and your osw file to use the desired weather file\nWe recommend using absolute paths for all commands, for cleaner output\n"

In [4]:
#output = `#{command} 2>&1`  #if there are errors and you need stderr

In [5]:
Dir.entries(".")

["..", ".", "URBANoptOSA", ".ipynb_checkpoints", "URBANopt_template.json", "create_URBANopt_OSA.ipynb", "create_URBANopt.ipynb", "submit_URBANopt.ipynb", "submit_single_run.ipynb"]

In [6]:
Dir.entries("URBANoptOSA")

["example_project.json", "mappers", "osm_building", "runner.conf", "Gemfile", "visualization", "..", "validation_schema.yaml", ".", "weather"]

In [7]:
command = "uo create --scenario-file #{Dir.pwd}/URBANoptOSA/example_project.json"

"uo create --scenario-file /examples/notebooks/URBANoptOSA/example_project.json"

In [8]:
output = `#{command}`

"\nBuilding sample ScenarioFiles, assigning mapper classes to each feature from example_project.json\n\nDone\n"

In [9]:
Dir.entries("URBANoptOSA")

["highefficiency_scenario.csv", "example_project.json", "evcharging_scenario.csv", "mappers", "thermalstorage_scenario.csv", "osm_building", "runner.conf", "Gemfile", "visualization", "baseline_scenario.csv", "..", "validation_schema.yaml", ".", "weather", "flexiblehotwater_scenario.csv"]

In [10]:
require 'csv'
file = CSV.table("#{Dir.pwd}/URBANoptOSA/highefficiency_scenario.csv")
CSV.open("#{Dir.pwd}/URBANoptOSA/highefficiency_scenario.csv", "wb") do |csv|
    csv << file[0]
    csv << file[1]
    csv << file[2]
end

#<CSV io_type:File io_path:"/examples/notebooks/URBANoptOSA/highefficiency_scenario.csv" encoding:ASCII-8BIT lineno:3 col_sep:"," row_sep:"\n" quote_char:"\"">

In [11]:
require 'fileutils'
FileUtils.mkdir_p "#{Dir.pwd}/OSA/lib/urbanopt"

["/examples/notebooks/OSA/lib/urbanopt"]

In [12]:
FileUtils.cp_r "#{Dir.pwd}/URBANoptOSA/.", "#{Dir.pwd}/OSA/lib/urbanopt"

In [13]:
require 'zip'

true

In [14]:
class ZipFileGenerator
  # Initialize with the directory to zip and the location of the output archive.
  def initialize(input_dir, output_file)
    @input_dir = input_dir
    @output_file = output_file
  end

  # Zip the input directory.
  def write
    entries = Dir.entries(@input_dir) - %w[. ..]

    ::Zip::File.open(@output_file, create: true) do |zipfile|
      write_entries entries, '', zipfile
    end
  end

  private

  # A helper method to make the recursion work.
  def write_entries(entries, path, zipfile)
    entries.each do |e|
      zipfile_path = path == '' ? e : File.join(path, e)
      disk_file_path = File.join(@input_dir, zipfile_path)

      if File.directory? disk_file_path
        recursively_deflate_directory(disk_file_path, zipfile, zipfile_path)
      else
        put_into_archive(disk_file_path, zipfile, zipfile_path)
      end
    end
  end

  def recursively_deflate_directory(disk_file_path, zipfile, zipfile_path)
    zipfile.mkdir zipfile_path
    subdir = Dir.entries(disk_file_path) - %w[. ..]
    write_entries subdir, zipfile_path, zipfile
  end

  def put_into_archive(disk_file_path, zipfile, zipfile_path)
    zipfile.add(zipfile_path, disk_file_path)
  end
end


:put_into_archive

In [15]:
zf = ZipFileGenerator.new("#{Dir.pwd}/OSA", "#{Dir.pwd}/URBANopt.zip")
zf.write()

["lib"]

In [16]:
RUBY_CMD = 'ruby'
BUNDLE_CMD = 'bundle exec ruby'
META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
PROJECT = File.absolute_path("#{Dir.pwd}")
HOST = '192.168.65.3'

"192.168.65.3"

In [17]:
@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD
@bundle_cmd = BUNDLE_CMD

"bundle exec ruby"

In [18]:
command = "#{@meta_cli} run_analysis --debug --verbose '#{@project}/URBANopt_template.json' 'http://#{@host}' -z 'URBANopt' -a single_run"

"/opt/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose '/examples/notebooks/URBANopt_template.json' 'http://192.168.65.3' -z 'URBANopt' -a single_run"

In [19]:
run_analysis = system(command)

true

In [20]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"5f206eb4-ed8d-440a-997e-6228c7e2821c"

In [ ]:
    status = 'queued'
    timeout_seconds = 360
    begin
      ::Timeout.timeout(timeout_seconds) do
        while status != 'completed'
          # get the analysis pages
          get_count = 0
          get_count_max = 12
          begin
            a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
            a = JSON.parse(a, symbolize_names: true)
            analysis_type = a[:analysis][:analysis_type]

            status = a[:analysis][:status]
            puts "Accessed pages for analysis: #{analysis_id}, analysis_type: #{analysis_type}, status: #{status}"
            jobs = a[:analysis][:jobs]
            puts "jobs: #{jobs}"

            a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
            a = JSON.parse(a, symbolize_names: true)
            status_message = a[:analysis][:status_message]
            puts "status_message: #{status_message}"

            # get all data points in this analysis
            a = RestClient.get "http://#{@host}/data_points.json"
            a = JSON.parse(a, symbolize_names: true)
            data_points = []
            a.each do |data_point|
              if data_point[:analysis_id] == analysis_id
                data_points << data_point
              end
            end
            # confirm that queueing is working
            data_points.each do |data_point|
              # get the datapoint pages
              data_point_id = data_point[:_id]

              a = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
              a = JSON.parse(a, symbolize_names: true)

              data_points_status = a[:data_point][:status]
              puts "Accessed pages for data_point #{data_point_id}, data_points_status = #{data_points_status}"
            end
          rescue RestClient::ExceptionWithResponse => e
            puts "rescue: #{e} get_count: #{get_count}"
            sleep(30)
            retry if get_count <= get_count_max
          end
          puts ''
          sleep(30)
        end
      end
    rescue ::Timeout::Error
      puts "Analysis status is `#{status}` after #{timeout_seconds} seconds; assuming error."
    end
    puts "Analysis status is `#{status}`"